# Linux

## Administrar Usuarios en Linux

* crear usuarios tstark, cdanvers, dprince en servidor
* crear grupo superhero
* hacer usuario tstark  un superusuario
* agregar grupo uperhero a los usuarios tstark, cdanvers, dprince
* bloquear cuenta de usuario dprince


In [ ]:
# abrir consola ssh Linux 

# entrar con usuario ssh
# usuario: cloud_user
# pass: pass
# usuario: cloud_user
# Public IP address of the Server1

ssh usuario@Public IP
password: pass# contraseña


In [ ]:
# agregar usuarios

useradd tstark
useradd dprince
useradd cdanvers 

In [ ]:
# crear grupo

groupadd superhero

In [ ]:
# cambiar grupo wheel a usuario tstark
usermod -g wheel tstark # id usuario

# agregar grupo superhero como grupo suplementario para todos los usuarios
usermod -aG grupo usuario

usermod -aG superhero tstark
usermod -aG superhero dprince
usermod -aG superhero cdanvers

# cada usuario tiene su propio grupo y pertenece al grupo superhero


In [ ]:
# bloquear usuario dprince
usermod -L dprince

## Crear usuarios sudo

* crear 2 usuarios 
* cambiar archivo /etc/sudoers
* probar accesos
* set administrador web 

In [ ]:
# crear usuario sudo
sudo useradd -m gfreeman

# agregar usuario avance
# asignar usuario avance a  grupo wheel suplementario
sudo useradd -G wheel -m avance

# set contraseña de los usuarios:  pass: LASudo321

sudo passwd gfreeman
pass: LASudo321

sudo passwd avance
pass: LASudo321


In [ ]:
# comprobar archivos /etc/sudoers permiten al grupo wheel acceso a los comandos sudo

sudo visudo

In [ ]:
# cambiar a cuenta avance, - para usar login de shell
sudo su - avance

# salir de la cuenta sudo
exit

In [ ]:
# configurar usuario gfreeman para webmaster
# crear nuevo archivo  sudoers en diccionario /etc/sudoers.d con entrada unica para webmasters

sudo visudo -f /etc/sudoers.d/web_admin
Cmnd_Alias  WEB = /bin/systemctl restart httpd.service, 
/bin/systemctl reload httpd.service

# permitir acceso a sudo a usuario gfreeman     
gfreeman ALL=WEB



In [ ]:
# entrar en cuenta gfreeman
sudo su - gfreeman

# intentar reiniciar 
sudo systemctl restart httpd.service

## Uso de Terminal SSH, redireccionamiento y Permisos de Linux

* permitir ssh sin pass de dev@server1 a dev@server2
* copiar todos los archivos .tar de  server1 a server2
* set umask para que los nuevos archivos sean read/write por su propietario solamente
* run el script 'deploy'


In [ ]:
# entrar a ssh con usuario dev y Public IP address
ssh usuario@Public IP address
pass: password servidor1


In [ ]:
# generar key
ssh-keygen

# copiar a  private IP de servidor2
ssh-copy-id  private IP servidor2

# entrar/loggin al servidor2 con IP publica
ssh Public IP servidor2
 

In [ ]:
# copiar todos los archivos .tar de servidor1 a servidor2 de  /home/dev/
scp *.gz server2_PRIVATE_IP:~/

In [ ]:
# ssh conectar a servidor2
ssh <server2_PRIVATE_IP>

# verificar que todos los archivos .tar desde servidor1 fueron copiados 
ll

In [ ]:
# extraer archivos .tar en servidor2
# >>: appending
# >: sobreescribir

tar -xvf deploy_content.tar.gz >> tar-output.log
tar -xvf deploy_script.tar.gz >> tar-output.log

# revisar directorio
ll


In [ ]:
# set unmask en servidor2 para que nuevos archivos sean ONLY READABLE and WRITEABLE por el propietario
umask 0066

# verificar permisos en deploy.sh
# hacer script ejecutable
chmod +x deploy.sh

# run archivo deploy.sh 
./deploy.sh


## Configurar el Boot Target por defecto

In [ ]:
## default target actual es en multi-user

# revisar que target actual es tipo multi-user
systemctl get-default

# cambiar default target para que el computedor arranque(boot) en desktop gráfico
sudo systemctl set-default graphical.target

# revisar que target actual es tipo graphical
systemctl get-default


## Agendar un servicio Systemd con Timer Units


In [ ]:

# entrar a ssh como usuario root
sudo su -

# check directorio actual
pwd

# ver recursos en el directorio
ls

# crear archivo timer unit 
vi web-backup.timer

# codigo de archivo timer unit

[Unit]
Description=Fire off the backup

[Timer]
OnCalendar=*-*-* 23:00:00
Persistent=true
Unit=web-backup.service

[Install]
WantedBy=multi-user.target

# salir de  archivo timer unit
### ESC
### :wq

In [ ]:
# copiar script shell en /usr/local/sbin
cp web-backup.sh /usr/local/sbin/

# copiar servicio y archivos timer en /etc/systemd/
cp web-backup.{service,timer} /etc/systemd/system/

# recargar el Systemd daemon para calcular las dependencias de servicio
systemctl daemon-reload

# habilitar servicios para correr un Boot
systemctl enable web-backup.service
systemctl enable web-backup.timer

# set permisos del archivo para ejecutar
chmod +x /usr/local/sbin/web-backup.sh

# iniciar servicios manualmente
systemctl start web-backup.timer web-backup.service

# check status de timer y servicio
systemctl status web-backup.timer
  Loaded: loaded (/etc/systemd/system/web-backup.timer; enabled; vendor preset: disabled)
   Active: active (waiting) since Mon 2023-08-14 10:33:07 EDT; 35s ago

systemctl status web-backup.service
● web-backup.service - Backup the web site, every day, or the boss will be mad
   Loaded: loaded (/etc/systemd/system/web-backup.service; enabled; vendor preset: disabled)
   Active: inactive (dead) since Mon 2023-08-14 10:33:07 EDT; 1min 17s ago
  Process: 4359 ExecStart=/usr/local/sbin/web-backup.sh (code=exited, status=0/SUCCESS)
 Main PID: 4359 (code=exited, status=0/SUCCESS)

# servidor listo para usarse


## Journal Control Utility: Administrar archivos y Troubleshooting


In [ ]:
# cambiar a cuenta root
sudo su -

# check el estado del servicio web
systemctl status httpd.service

# intentar iniciar el servicio web
systemctl start httpd.service

# check Journal
journalctl -u httpd.service

-- Logs begin at Mon 2023-08-14 10:10:57 EDT, end at Mon 2023-08-14 10:43:37 EDT. --
Aug 14 10:43:37 ip-10-0-1-102.ec2.internal systemd[1]: Starting The Apache HTTP Server...
Aug 14 10:43:37 ip-10-0-1-102.ec2.internal httpd[22861]: httpd: Could not open configuration fi
Aug 14 10:43:37 ip-10-0-1-102.ec2.internal systemd[1]: httpd.service: main process exited, code
Aug 14 10:43:37 ip-10-0-1-102.ec2.internal systemd[1]: Failed to start The Apache HTTP Server.
Aug 14 10:43:37 ip-10-0-1-102.ec2.internal systemd[1]: Unit httpd.service entered failed state.
Aug 14 10:43:37 ip-10-0-1-102.ec2.internal systemd[1]: httpd.service failed.

# revisar directorio donde la configuracion httpd debe estar
ls /etc/httpd/conf

httpd.conf.bkup  magic

# restaurar la configuracion httpd original del archivo  
mv /etc/httpd/conf/httpd.conf.bkup /etc/httpd/conf/httpd.conf

# reiniciar el servicio
systemctl restart httpd.service

# check el estado del servicio
systemctl status httpd.service

● httpd.service - The Apache HTTP Server
   Loaded: loaded (/usr/lib/systemd/system/httpd.service; disabled; vendor preset: disabled)
   Active: active (running) since Mon 2023-08-14 10:49:02 EDT; 39s ago
     Docs: man:httpd(8)
           man:apachectl(8)
 Main PID: 22879 (httpd)
   Status: "Total requests: 0; Current requests/sec: 0; Current traffic:   0 B/sec"
   CGroup: /system.slice/httpd.service
           ├─22879 /usr/sbin/httpd -DFOREGROUND
           ├─22880 /usr/sbin/httpd -DFOREGROUND
           ├─22881 /usr/sbin/httpd -DFOREGROUND
           ├─22882 /usr/sbin/httpd -DFOREGROUND
           ├─22883 /usr/sbin/httpd -DFOREGROUND
           └─22884 /usr/sbin/httpd -DFOREGROUND

# navegar a pagina web local 
elinks http://localhost

## Generar y cambiar SSH Keys para acceso seguro

### Crear ssh keys en servidor1 y servidor2

In [ ]:
# ingresar servidor1
ssh usuario@ IP_servidor1

# ver archivos de directorio
ls -la

# cambiar directorio a .ssh
cd .ssh

# ver archivos de directorio
ls -la

# generar ssh key para servidor1
ssh-keygen
# ver archivos de directorio
ls -la

# ver contenido de archivo  id_rsa.pub
cat id_rsa.pub


# ingresar servidor2
ssh usuario@ IP_servidor2

# cambiar directorio a .ssh
cd .ssh

# ver archivos de directorio
ls -la

# instalar editor de texto nano
sudo yum install nano
pass: password

# abrir archivo  authorized_keys
nano authorized_keys

# agregar key

control+x
yes


### Cambiar ssh key entre servidores

In [ ]:
# en servidor2
ssh-keygen

# ver archivos de directorio
ls -la

# ver contenido de archivo id_rsa.pub
cat id_rsa.pub

# copiar key
# salir de servidor2
exit

# instalar editor de texto nano en servidor
sudo yum install nano

# ver archivos de directorio
ls -la

# abrir archivo authorized_keys en archivo de texto nano 
nano authorized_keys



### Probar configuración

In [ ]:
# entrar a servidor2 desde servidor1
ssh usuario@ IP_servidor2

# entrar a servidor1 desde servidor2
ssh usuario@ IP_servidor1



## Crear y usar SSH tunel para tráfico de red

### ssh para cliente a servidor sin contraseña

In [ ]:
# ingresar a servidor CLIENT
ssh usuario@IP_servidor CLIENT

# generar key
ssh-keygen
ENTER

# copiar key de SSH Tunnel Server
ssh-copy-id cloud_user@IP_servidor Tunnel (privada)

# salir de servidor Tunnel
logout


### verificar accesibildad a ssh Tunnel 

In [ ]:
# crear SSH Tunnel
ssh -f cloud_user@10.0.1.100 -L 2000:10.0.1.100:80 -N

# usar comando curl para verificar accesibilidad
curl localhost:2000 

Webpage worked (check)

## Administrar seguridad con TCP Wrappers y Systemd Sockets



### Configurar sshd para usar Sockets

In [ ]:
# entrar a servidor
ssh cloud_user@IP_servidor

# verificar que unidad sshd.socket no esta activa
systemctl status sshd.socket


● sshd.socket - OpenSSH Server Socket
   Loaded: loaded (/usr/lib/systemd/system/sshd.socket; disabled; vendor preset: disabled)
   Active: inactive (dead)
     Docs: man:sshd(8)
           man:sshd_config(5)
   Listen: [::]:22 (Stream)
 Accepted: 0; Connected: 0


# set 'at job' que detiene la unidad sshd.service, iniciar sshd.socket
sudo at now + 3 minutes
pass: password

# agregar
at> systemctl stop sshd.service
at> systemctl start sshd.socket

# terminar configuracion 'at job'
control+D

# verificar que unidad sshd.socket esta activa y funcionando
systemctl status sshd.socket

# habilitar socket para SSH y deshabilitar servicio para SSH
sudo systemctl enable sshd.socket
sudo systemctl disable sshd.service

### Set TCP Wrapper para permitir solo SSH


In [ ]:
# verificar que el servidor sshd se compiló para usar TCP wrappers
ldd /usr/sbin/sshd | grep libwrap

# editar el archivo /etc/hosts.allow
sudo vim /etc/hosts.allow


# agregar linea de codigo al archivo
sshd2 sshd : ALL

# editar archivo /etc/hosts.deny
sudo vim /etc/hosts.deny

# agregar linea de codigo al archivo
ALL : ALL

# salir sesión ssh
exit

# reconectar a la sesión segura shell
ssh cloud_user@&IP_servidor



## Instalar y Administrar paquetes en Debian/Ubuntu

### Instalar paquetes en servidor web Apache

In [ ]:

# actualizar paquetes
sudo apt update

# instalar paquetes
sudo apt install apache2 wget



### Verificar que el servidor está funcionando y ver resultado


In [ ]:
# verificar servidor
curl http://localhost

# capturar salida de http request
wget --output-document=local_index.response http://localhost (local_index.response)

# quitar local_index.response
rm local_index.response

# mover index.html a  local_index.response
mv index.html local_index.response




## Instalar y Administrar paquetes en Red Hat/CentOS 

### Instalar aplicaciones E-links disponibles

In [ ]:

# entrar a servidor publico 
ssh usuario@IP_servidor

# ver archivos de directorio
ls -la

# ver información de carpeta 'Downloads'
Downloads/

# cambiar a directorio Downloads
cd Downloads/

# ver archivos de directorio
ls -la

# instalar el paquete e-links RPM
sudo rpm -i elinks-0.12-0.37.pre6.el7.0.1.x86_64.rpm
pass: password

# ver dependencia que falla y ver qué provee la dependencia
yum provides libmozjs185*

# instalar el paquete js
sudo yum install js
y

# intentar instalar el paquete e-links RPM
sudo rpm -i elinks-0.12-0.37.pre6.el7.0.1.x86_64.rpm

# ver dependencia que falla y ver qué provee la dependencia
yum provides */libnss_compat_ossl*

# instalar nss_compat_ossl
sudo yum install nss_compat_ossl

# intentar instalar el paquete e-links RPM
sudo rpm -i elinks-0.12-0.37.pre6.el7.0.1.x86_64.rpm

### Verificar paquetes E-links RPM existen

In [ ]:
# ejectutar E-links
elinks

# abrir website por URL
http://www.amazon.com

## Troubleshooting RPM

### Instalar telnet

In [ ]:
# entrar a servidor publico 
ssh usuario@IP_servidor

# convertirse en usuario root
sudo -i

# instalar paquete telnet
yum install -y telnet

# verificar integridad 
cd /var/lib/rpm/
/usr/lib/rpm/rpmdb_verify Packages

# mover carpeta 'Packages' a 'Packages.bad' y crear nueva base de datos RPM de 'Packages.bad'
mv Packages Packages.bad
/usr/lib/rpm/rpmdb_dump Packages.bad | /usr/lib/rpm/rpmdb_load Packages

# verificar integridad de nueva base de datos RPM
/usr/lib/rpm/rpmdb_verify Packages

# Query los pquetes instalados para errores
rpm -qa > /dev/null

# reconstruir la base de datos RPM
rpm -vv --rebuilddb

# instalar telnet
yum install -y telnet



### Actualizar Apache

In [ ]:
# intentar instalar Apache
yum install -y httpd

# editar el archivo /etc/yum.conf
vim /etc/yum.conf

# quitar la exclusión para httpd
exclude=httpd

# guardar y cerrar archivo
:wq 

# instalar Apache
yum install -y httpd

## Crear Estructura de Directorio en Linux

### Crear Directorio Padre

In [ ]:
# entrar a servidor  publico
ssh usuario@IP_servidor

# manual
mkdir -p Projects/ancient
mkdir Projects/classical
mkdir Projects/medieval

# expasion bash
mkdir -p Projects/{ancient,classical,medieval}

### Crear Sub-directorios

In [ ]:
# manual
mkdir Projects/ancient/egyptian
mkdir Projects/ancient/nubian
mkdir Projects/classical/greek
mkdir Projects/medieval/britain
mkdir Projects/medieval/japan


# expasion bash
mkdir Projects/ancient/{egyptian,nubian}
mkdir Projects/classical/greek
mkdir Projects/medieval/{britain,japan}


### Crear archivos vacios

In [ ]:
# crear archivos vacios
touch Projects/ancient/nubian/further_research.txt
touch Projects/classical/greek/further_research.txt



### Cambiar nombre de sub-directorio 

In [ ]:

mv Projects/classical Projects/greco-roman

## Archivos .Tar/Archivos Comprimidos

In [ ]:
# ingresar a servidor 

ssh usuario@IP_servidor

### Tamaño de archivo .txt original

In [ ]:
# ver tamaño original de archivo
ls -lh junk.txt

### Crear archivos .zip


In [ ]:
# comprimir archivo con gzip
gzip junk.txt

# ver tamaño de archivo
ls -lh

# descomprimir archivo .gzip
gunzip junk.txt.gz


# comprimir archivo con bzip2
bzip2 junk.txt

# ver tamaño de archivo
ls -lh junk.txt.bz2

# descomprimir archivo .gzip
bunzip2 junk.txt.bz2




# comprimir archivo con xz
xz junk.txt

# ver tamaño de archivo
ls -lh

# descomprimir archivo .gzip
unxz junk.txt.xz




### Crear archivos .tar

In [ ]:
# crear tarball con gzip
tar -cvzf gztar.tar.gz junk.txt

# crear tarball con bzip2
tar -cvjf bztar.tar.bz2 junk.txt

# crear tarball con xz
tar -cvJf xztar.tar.xz junk.txt

# comprar tamaños de archivos comprimidos
ls -lh


### Leer archivos comprimidos (texto)

In [ ]:
# copiar archivo /etc/passwd a directorio principal (home)
cp /etc/passwd /home/cloud_user/

# comprimir archivo .tar con gzip
tar -cvzf passwd.tar.gz passwd

# leer archivo comprimido 
zcat passwd.tar.gz

# comprimir arhivo a tarball con bzip2
tar -cvjf passwd.tar.bz2 passwd

# leer archivo comprimido 
bzcat passwd.tar.bz2

# comprimir arhivo a tarball con xz
tar -cvJf passwd.tar.xz passwd

# leer archivo comprimido
xzcat passwd.tar.xz


## Administrar atributos de archivos y Permisos

### Dar acceso al Directorio

In [ ]:
# entrar a servidor
ssh usuario@IP_servidor

# cambiar a directorio opt
cd /opt

# ver archivos y permisos de directorio
ls -la

# intentar acceder a directorio myapp
cd myapp/


# permisos de acceso solo para usuario  tkirk
# cambiar permisos de acceso
sudo chmod 777 myapp

# ver archivos y permisos de directorio
ls -la

# intentar acceder a directorio myapp con nuevos permisos
cd myapp/


### Cambiar **Permisos** del Directorio

In [ ]:
# quitar permisos de ejecucción(ejecutar)
sudo chmod -f -x -R *

# dar permisos de leer y escribir (read/write)
# también quitar permisos de ejecucción automáticamente
sudo chmod 666 -f -R *

# ver archivos y permisos de directorio
ls -la

# NOTA: para que los usuarios puedan navegar en los directorios, los directorios deben estar en ejecutable
sudo find /opt/myapp -type d -exec chmod o+x {} \;


### 